![title](kctdeploy.JPG)

# DEPLOY  TEAM  KGT

# NORTH SEATTLE COLLEGE GREETING CHATBOT

## Team Kevin, Thomas and Captain

## Notebook, Flask and Deploy a text model with NSC chatbots

## STEP IMPORTS

In [22]:
import numpy as np
import tflearn
import tensorflow as tf
import random
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

## LOAD DATA

In [25]:
import pickle
data = pickle.load( open( "models/training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

import json
with open('data/NSCints.json') as json_data:
    intents = json.load(json_data)

## TAKE FIRST LOOK

In [3]:
def ngrams(str, n):
    tokens = str.split(' ')
    arr = []
    for i in range(len(tokens)):
        new_str = ''
        if i == 0 and n>1:
            new_str = '_'
            for j in range(n):
                if j < n - 1:
                    if (i + j) <= len(tokens):
                        new_str += ' '+tokens[i+j]
                    else:
                        new_str += ' _'
        else:
            for j in range(n):
                if j < n:
                    if (i + j) < len(tokens):
                        if j == 0:
                            new_str += tokens[i+j]
                        else:
                            new_str += ' '+tokens[i+j]
                    else:
                        new_str += ' _'
        arr.append(new_str)
    return arr

In [4]:
ngrams('a b c d e f g h', 4)

['_ a b c',
 'b c d e',
 'c d e f',
 'd e f g',
 'e f g h',
 'f g h _',
 'g h _ _',
 'h _ _ _']

In [26]:
words = []
classes = []
documents = []
ignore_words = ['?', 'và', 'à', 'ừ', 'ạ', 'vì', 'từng', 'một_cách']

for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

26 documents
8 classes ['classes', 'enroll', 'goodbye', 'greeting', 'hours', 'open', 'payments', 'thanks']
50 unique stemmed words ['about', 'acceiv', 'aid', 'al', 'am', 'anyon', 'ar', 'bye', 'cash', 'class', 'day', 'do', 'enrol', 'fin', 'good', 'goodby', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'in', 'is', 'it', 'lik', 'mastercard', 'nsc', 'o.k', 'on', 'op', 'or', 'program', 'season', 'set', 'start', 'term', 'thank', 'the', 'ther', 'thi', 'think', 'to', 'vis', 'want', 'what', 'when', 'wil', 'yo', 'you']


## TRAINNING

In [6]:

training = []
output = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [7]:
print(train_x[1])
print(train_y[1])

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0]


## Analyze and Clean Data

In [8]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [9]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)

    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# bag of words
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [10]:
bow('I want to join NSC', words)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0])

## PREDICTIONS

### Load Model

In [11]:
model.load('./models/model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Captain\Documents\WINTER2020\AD470\LAB\ANACON10\models\model.tflearn


In [12]:
# data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='1', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        return print(random.choice(i['responses']))

            results.pop(0)

In [13]:
classify('is North Seattle open today?')

[('greeting', 0.5746683)]

In [14]:
response('is North Seattle open today?')

Hi there NSC student, how can I help?


In [15]:
response('do you take cash?')

NSC accept cards, checks and has financial aid


In [16]:
response('Goodbye, see you later')

Have a nice very NSC day


In [17]:
response('I want join classes')

enroll en more than 1000 classes at NSC


In [18]:
classify('Hi, I am a student')

[('greeting', 0.44897944), ('thanks', 0.36532196)]

In [19]:
response('Hi, I am a student')

Good to see you again here in NSC


In [20]:
response('thank you, I want to join NSC')

Please enroll as soon as possible


## A BETTER MODEL

In [21]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('models/model.tflearn')

Training Step: 3999  | total loss: 0.06187 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.06187 - acc: 0.9995 -- iter: 24/26
Training Step: 4000  | total loss: 0.06132 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.06132 - acc: 0.9995 -- iter: 26/26
--
INFO:tensorflow:C:\Users\Captain\Documents\WINTER2020\AD470\LAB\ANACON10\models\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
